<a href="https://colab.research.google.com/github/tinayiluo0322/ECE_Final_Project_2025/blob/main/Models/2_Class_Garbage_Data_ResNet18_Feature_Extraction_Transfer_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import sys
import os
import random
import numpy as np
import torch

In [3]:
import torch.backends.cudnn as cudnn
import torch.nn as nn
import torch.nn.functional as F
import os
import numpy as np
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset, Dataset, random_split, WeightedRandomSampler
from sklearn.model_selection import train_test_split
from collections import Counter
import shutil
from PIL import Image
import torch.optim as optim
from torch.optim import lr_scheduler
import time
import copy
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from tqdm.notebook import tqdm

In [4]:
seed = 42
random.seed(seed)  # Python's random module
np.random.seed(seed)  # NumPy's random module
torch.manual_seed(seed)  # PyTorch's random seed for CPU
torch.cuda.manual_seed(seed)  # PyTorch's random seed for the current GPU
torch.cuda.manual_seed_all(seed)  # PyTorch's random seed for all GPUs (if using multi-GPU)

# Ensure deterministic behavior on GPU (optional, may slow down training)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Optional: Set environment variables for further reproducibility
os.environ['PYTHONHASHSEED'] = str(seed)

In [5]:
# check if CUDA is available
train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
  print('CUDA is not available.  Training on CPU ...')
else:
  print('CUDA is available!  Training on GPU ...')

CUDA is available!  Training on GPU ...


## Load the pretrained ResNet18 model with CIFAR-10 data

In [6]:
# Step 1: Re-declare BasicBlock and ResNet
class BasicBlock(nn.Module):
  expansion = 1
  def __init__(self, in_planes, planes, stride=1):
    super(BasicBlock, self).__init__()
    self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
    self.bn1 = nn.BatchNorm2d(planes)
    self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
    self.bn2 = nn.BatchNorm2d(planes)

    self.shortcut = nn.Sequential()
    if stride != 1 or in_planes != self.expansion*planes:
      self.shortcut = nn.Sequential(
          nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=False),
          nn.BatchNorm2d(self.expansion*planes)
      )

  def forward(self, x):
    out = F.relu(self.bn1(self.conv1(x)))
    out = self.bn2(self.conv2(out))
    out += self.shortcut(x)
    out = F.relu(out)
    return out

class BottleNeck(nn.Module):
  expansion = 4

  def __init__(self, in_planes, planes, stride=1):
    super(BottleNeck, self).__init__()
    self.conv1 = nn.Conv2d(in_planes , planes, kernel_size=1, bias=False)
    self.bn1 = nn.BatchNorm2d(planes)
    self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
    self.bn2 = nn.BatchNorm2d(planes)
    self.conv3 = nn.Conv2d(planes, self.expansion*planes, kernel_size=1, bias=False)
    self.bn3 = nn.BatchNorm2d(self.expansion*planes)

    self.shortcut = nn.Sequential()
    if stride != 1 or in_planes != self.expansion*planes :
      self.shortcut = nn.Sequential(
          nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=False),
          nn.BatchNorm2d(self.expansion*planes)
      )

  def forward(self, x):
    out = F.relu(self.bn1(self.conv1(x)))
    out = F.relu(self.bn2(self.conv2(out)))
    out = self.bn3(self.conv3(out))
    out += self.shortcut(x)
    out = F.relu(out)
    return out

class ResNet(nn.Module):
  def __init__(self, block, num_blocks, num_classes=10):
    super(ResNet, self).__init__()
    self.in_planes = 64

    self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
    self.bn1 = nn.BatchNorm2d(64)
    self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
    self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
    self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
    self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
    self.linear = nn.Linear(512*block.expansion, num_classes)

  def _make_layer(self, block, planes, num_blocks, stride):
    strides = [stride] + [1]*(num_blocks-1)
    layers = []
    for stride in strides:
      layers.append(block(self.in_planes, planes, stride))
      self.in_planes = planes * block.expansion
    return nn.Sequential(*layers)

  def forward(self, x):
    out = F.relu(self.bn1(self.conv1(x)))
    out = self.layer1(out)
    out = self.layer2(out)
    out = self.layer3(out)
    out = self.layer4(out)
    out = F.avg_pool2d(out, 4)
    out = out.view(out.size(0), -1)
    out = self.linear(out)
    return out

In [7]:
# # Step 2: Instantiate the model
# model = ResNet(BasicBlock, [2, 2, 2, 2])

# # Step 3: Load the pretrained weights correctly
# # Define device first!
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# pretrained_path = "/content/drive/MyDrive/ECE661 final project/ResNet18.pt"
# state_dict = torch.load(pretrained_path, map_location=device)

# # If trained with DataParallel, remove 'module.' prefix
# from collections import OrderedDict

# new_state_dict = OrderedDict()
# for k, v in state_dict.items():
#     name = k[7:] if k.startswith('module.') else k  # strip 'module.' if present
#     new_state_dict[name] = v

# model.load_state_dict(new_state_dict)

# # Step 4: Move model to device and eval mode
# model = model.to(device)
# model.eval()

# print("Pretrained ResNet18 loaded and ready!")

In [8]:
# Step 2: Instantiate the model
model = ResNet(BasicBlock, [2, 2, 2, 2])

# Use DataParallel
if train_on_gpu:
  model = torch.nn.DataParallel(model)
  cudnn.benchmark = True

# Step 3: Load the pretrained weights
# (make sure ResNet18.pt is in your working directory or give full path)
pretrained_path = "/content/drive/MyDrive/ECE661 final project/ResNet18.pt"
model.load_state_dict(torch.load(pretrained_path, map_location=torch.device('cuda' if torch.cuda.is_available() else 'cpu')))

# Step 4: Move model to device and eval mode
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
model.eval()

print("Pretrained ResNet18 loaded and ready!")

Pretrained ResNet18 loaded and ready!


## Data Processing of 2 class Garbage Data (Data Augmentation + Weighted sampler for class balancing)

- Data Augmentation: Random Horizontal Flip, Random Rotation, Color Jitter

- Weighted Sampling: Without any weighting, each sample has an equal probability of being chosen during training. With weighted sampling, we artificially adjust these probabilities to oversample the minority class and undersample the majority class.

In [9]:
# Paths and config
DATA_PATH = "/content/drive/MyDrive/two_class_garbage-dataset"
batch_size = 32
num_workers = 2
valid_size = 0.2
test_size = 0.2

# CIFAR-10 normalization stats (correct for your pretrained model)
mean = (0.4914, 0.4822, 0.4465)
std = (0.2023, 0.1994, 0.2010)

# Custom Dataset class for handling transforms properly
class TransformedSubset(Dataset):
    def __init__(self, subset, transform=None):
        self.subset = subset
        self.transform = transform

    def __getitem__(self, idx):
        x, y = self.subset[idx]
        if self.transform:
            x = self.transform(x)
        return x, y

    def __len__(self):
        return len(self.subset)

# Define transforms - important to resize to 32x32 to match CIFAR-10
transform_train = transforms.Compose([
    transforms.Resize((32, 32)),  # Resize to CIFAR-10 dimensions
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean, std),
])

transform_test = transforms.Compose([
    transforms.Resize((32, 32)),  # Resize to CIFAR-10 dimensions
    transforms.ToTensor(),
    transforms.Normalize(mean, std),
])

# First create the dataset normally
dataset = datasets.ImageFolder(
    DATA_PATH,
    transform=None
)

# Then define your custom mapping
class_to_idx = {
    'non_battery': 0,  # Make non-battery class 0
    'battery': 1       # Make battery class 1
}

# Update the dataset's class_to_idx and rebuild the samples list
original_to_new = {v: class_to_idx[k] for k, v in dataset.class_to_idx.items()}
dataset.class_to_idx = class_to_idx
dataset.samples = [(path, original_to_new[idx]) for path, idx in dataset.samples]
dataset.targets = [original_to_new[idx] for idx in dataset.targets]

# Verify the mapping
print("Class names:", dataset.classes)
print("Class to index mapping:", dataset.class_to_idx)

# Extract targets for stratification
targets = np.array([sample[1] for sample in dataset.samples])

# Print class distribution to see imbalance
class_counts = Counter(targets)
print("Class distribution:", dict(class_counts))
imbalance_ratio = max(class_counts.values()) / min(class_counts.values())
print(f"Imbalance ratio: {imbalance_ratio:.2f}:1")

# Stratified split: train+val vs test
train_val_idx, test_idx = train_test_split(
    np.arange(len(targets)),
    test_size=test_size,
    stratify=targets,
    random_state=42,
)

# Stratified split: train vs val
train_idx, val_idx = train_test_split(
    train_val_idx,
    test_size=valid_size / (1 - test_size),
    stratify=targets[train_val_idx],
    random_state=42,
)

# Create proper transformed subsets
train_set = TransformedSubset(Subset(dataset, train_idx), transform_train)
val_set = TransformedSubset(Subset(dataset, val_idx), transform_test)
test_set = TransformedSubset(Subset(dataset, test_idx), transform_test)

# Weighted sampler for class balancing
train_targets = targets[train_idx]
class_sample_counts = Counter(train_targets)
weights = 1. / np.array([class_sample_counts[t] for t in train_targets])
sampler = WeightedRandomSampler(weights, len(weights), replacement=True)

# DataLoaders
train_loader = DataLoader(train_set, batch_size=batch_size, sampler=sampler, num_workers=num_workers)
val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False, num_workers=num_workers)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False, num_workers=num_workers)

# Print dataset stats
print("Train size:", len(train_set))
print("Validation size:", len(val_set))
print("Test size:", len(test_set))
print("Train class distribution:", dict(Counter(train_targets)))
print("Using weighted sampler to balance training batches.")

# Function to verify balanced sampling
def check_batch_distribution(loader, num_batches=10):
    class_counts = Counter()
    for i, (_, labels) in enumerate(loader):
        if i >= num_batches:
            break
        class_counts.update(labels.numpy())
    print("Sampled batch distribution:", dict(class_counts))

print("Checking if WeightedRandomSampler is properly balancing batches:")
check_batch_distribution(train_loader)

Class names: ['battery', 'non_battery']
Class to index mapping: {'non_battery': 0, 'battery': 1}
Class distribution: {np.int64(1): 944, np.int64(0): 18786}
Imbalance ratio: 19.90:1
Train size: 11838
Validation size: 3946
Test size: 3946
Train class distribution: {np.int64(0): 11272, np.int64(1): 566}
Using weighted sampler to balance training batches.
Checking if WeightedRandomSampler is properly balancing batches:
Sampled batch distribution: {np.int64(0): 166, np.int64(1): 154}


## Feature Extraction Transfer Learning

In [10]:
# # Step 1: Modify the model for feature extraction
# # First, save the original classifier for reference
# original_classifier = model.linear

# # Replace the final linear layer with a new one for binary classification
# num_features = model.linear.in_features
# model.linear = nn.Linear(num_features, 2)  # 2 classes (battery/non-battery)

# # Freeze all layers except the final classifier
# for param in model.parameters():
#     param.requires_grad = False

# # Unfreeze the parameters of the final classifier
# for param in model.linear.parameters():
#     param.requires_grad = True

# # Move model to device
# model = model.to(device)

In [11]:
# Step 1: Modify the model for feature extraction
# Check if model is wrapped in DataParallel
if hasattr(model, 'module'):
    # First, save the original classifier for reference
    original_classifier = model.module.linear

    # Get number of features from the original classifier
    num_features = model.module.linear.in_features

    # Replace the final linear layer with a new one
    model.module.linear = nn.Linear(num_features, 2)  # 2 classes (battery/non-battery)

    # Freeze all layers except the final classifier
    for param in model.parameters():
        param.requires_grad = False

    # Unfreeze the parameters of the final classifier
    for param in model.module.linear.parameters():
        param.requires_grad = True
else:
    # First, save the original classifier for reference
    original_classifier = model.linear

    # Get number of features from the original classifier
    num_features = model.linear.in_features

    # Replace the final linear layer with a new one
    model.linear = nn.Linear(num_features, 2)  # 2 classes (battery/non-battery)

    # Freeze all layers except the final classifier
    for param in model.parameters():
        param.requires_grad = False

    # Unfreeze the parameters of the final classifier
    for param in model.linear.parameters():
        param.requires_grad = True

# Make sure the model is on the correct device
model = model.to(device)

In [12]:
# # Step 2: Define loss function, optimizer and metrics
# criterion = nn.CrossEntropyLoss()
# optimizer = optim.SGD(model.linear.parameters(), lr=0.001, momentum=0.9)
# scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

In [13]:
# Step 2: Define loss function, optimizer and metrics
criterion = nn.CrossEntropyLoss()

# Check if model is wrapped in DataParallel
if hasattr(model, 'module'):
    # Get parameters of the classifier from the module
    optimizer = optim.SGD(model.module.linear.parameters(), lr=0.001, momentum=0.9)
else:
    # Get parameters directly if not using DataParallel
    optimizer = optim.SGD(model.linear.parameters(), lr=0.001, momentum=0.9)

scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

In [14]:
def time_since(start_time):
    elapsed = time.time() - start_time
    return f"{int(elapsed // 60)}m {int(elapsed % 60)}s"

In [15]:
# Step 3: Feature Extraction Transfer Learning
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs-1}')
        print('-' * 10)

        # Then add a timestamp print at the beginning of each epoch:
        epoch_start = time.time()
        print(f"Starting epoch {epoch} at {time.strftime('%H:%M:%S')}")

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
                dataloader = train_loader
            else:
                model.eval()   # Set model to evaluate mode
                dataloader = val_loader

            running_loss = 0.0
            running_corrects = 0
            all_labels = []
            all_preds = []
            all_probs = []

            # Add progress bar for each phase
            progress_bar = tqdm(dataloader, desc=f"{phase}")

            # Iterate over data with progress bar
            for inputs, labels in progress_bar:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # Zero the parameter gradients
                optimizer.zero_grad()

                # Forward
                # Track history only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    probs = nn.functional.softmax(outputs, dim=1)[:, 1]  # Prob for class 1 (battery)
                    loss = criterion(outputs, labels)

                    # Backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # Statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

                # Collect for metrics
                all_labels.extend(labels.cpu().numpy())
                all_preds.extend(preds.cpu().numpy())
                all_probs.extend(probs.detach().cpu().numpy())

                # Update progress bar with current loss
                progress_bar.set_postfix(loss=f"{loss.item():.4f}")

            if phase == 'train' and scheduler is not None:
                scheduler.step()

            epoch_loss = running_loss / len(dataloader.dataset)
            epoch_acc = running_corrects.double() / len(dataloader.dataset)

            # Calculate additional metrics
            all_labels = np.array(all_labels)
            all_preds = np.array(all_preds)
            all_probs = np.array(all_probs)

            conf_matrix = confusion_matrix(all_labels, all_preds)

            # Calculate metrics for imbalanced data
            try:
                auc = roc_auc_score(all_labels, all_probs)
                tn, fp, fn, tp = conf_matrix.ravel()
                sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0  # Recall for positive class (battery)
                specificity = tn / (tn + fp) if (tn + fp) > 0 else 0  # Recall for negative class
                precision = tp / (tp + fp) if (tp + fp) > 0 else 0 # The proportion of predicted battery items that are actually batteries
                f1 = 2 * (precision * sensitivity) / (precision + sensitivity) if (precision + sensitivity) > 0 else 0
            except:
                auc, sensitivity, specificity, precision, f1 = 0, 0, 0, 0, 0

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f} AUC: {auc:.4f}')
            print(f'{phase} Sens: {sensitivity:.4f} Spec: {specificity:.4f} Prec: {precision:.4f} F1: {f1:.4f}')
            print(f'{phase} Confusion Matrix:\n{conf_matrix}')

            # Deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        # And at the end of each epoch:
        print(f"Epoch {epoch} completed in {time_since(epoch_start)}")

        print()

    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc:.4f}')

    # Load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [16]:
# Step 4: Execute the training
feature_extraction_model = train_model(model, criterion, optimizer, scheduler, num_epochs=5)

Epoch 0/4
----------
Starting epoch 0 at 14:13:03


train:   0%|          | 0/370 [00:00<?, ?it/s]

train Loss: 0.5487 Acc: 0.7249 AUC: 0.8024
train Sens: 0.7170 Spec: 0.7329 Prec: 0.7326 F1: 0.7247
train Confusion Matrix:
[[4294 1565]
 [1692 4287]]


val:   0%|          | 0/124 [00:00<?, ?it/s]

val Loss: 0.5536 Acc: 0.7086 AUC: 0.8534
val Sens: 0.8095 Spec: 0.7035 Prec: 0.1208 F1: 0.2102
val Confusion Matrix:
[[2643 1114]
 [  36  153]]
Epoch 0 completed in 59m 13s

Epoch 1/4
----------
Starting epoch 1 at 15:12:17


...

val Loss: 0.5247 Acc: 0.7377 AUC: 0.8726
val Sens: 0.8148 Spec: 0.7338 Prec: 0.1334 F1: 0.2293
val Confusion Matrix:
[[2757 1000]
 [  35  154]]
Epoch 3 completed in 7m 19s

Epoch 4/4
----------
Starting epoch 4 at 15:49:30


train:   0%|          | 0/370 [00:00<?, ?it/s]

train Loss: 0.4783 Acc: 0.7729 AUC: 0.8534
train Sens: 0.7551 Spec: 0.7908 Prec: 0.7834 F1: 0.7690
train Confusion Matrix:
[[4676 1237]
 [1451 4474]]


val:   0%|          | 0/124 [00:00<?, ?it/s]

val Loss: 0.4519 Acc: 0.7940 AUC: 0.8807
val Sens: 0.7884 Spec: 0.7943 Prec: 0.1616 F1: 0.2682
val Confusion Matrix:
[[2984  773]
 [  40  149]]
Epoch 4 completed in 4m 39s

Training complete in 101m 6s
Best val Acc: 0.7940


In [17]:
# Step 5: Evaluate on test set
def evaluate_model(model, test_loader):
    model.eval()
    all_labels = []
    all_preds = []
    all_probs = []

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            probs = nn.functional.softmax(outputs, dim=1)[:, 1]  # Prob for class 1 (battery)

            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(preds.cpu().numpy())
            all_probs.extend(probs.cpu().numpy())

    # Convert to numpy arrays
    all_labels = np.array(all_labels)
    all_preds = np.array(all_preds)
    all_probs = np.array(all_probs)

    # Calculate metrics
    conf_matrix = confusion_matrix(all_labels, all_preds)
    class_report = classification_report(all_labels, all_preds, target_names=['non_battery', 'battery'])
    auc = roc_auc_score(all_labels, all_probs)

    tn, fp, fn, tp = conf_matrix.ravel()
    sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0

    print("Test Set Evaluation:")
    print(f"Confusion Matrix:\n{conf_matrix}")
    print(f"Classification Report:\n{class_report}")
    print(f"AUC: {auc:.4f}")
    print(f"Sensitivity (Recall for battery class): {sensitivity:.4f}")
    print(f"Specificity (Recall for non-battery class): {specificity:.4f}")

In [18]:
# Run evaluation on test set
evaluate_model(feature_extraction_model, test_loader)

Test Set Evaluation:
Confusion Matrix:
[[2940  817]
 [  38  151]]
Classification Report:
              precision    recall  f1-score   support

 non_battery       0.99      0.78      0.87      3757
     battery       0.16      0.80      0.26       189

    accuracy                           0.78      3946
   macro avg       0.57      0.79      0.57      3946
weighted avg       0.95      0.78      0.84      3946

AUC: 0.8745
Sensitivity (Recall for battery class): 0.7989
Specificity (Recall for non-battery class): 0.7825


The model has a high false positive rate - it's classifying many non-batteries as batteries. This explains why precision for the battery class is so low (16%). However, it does well at identifying actual batteries (80% recall).

This behavior is somewhat expected with highly imbalanced datasets. The balanced sampling during training helped the model become sensitive to the minority class, but it resulted in many false positives.

In [19]:
# Save the model
torch.save(feature_extraction_model.state_dict(), '/content/drive/MyDrive//ECE661 final project/feature_extraction_model.pt')